In [1]:
import tensorflow as tf 
import tensorboard 
tf.compat.v1.disable_eager_execution()

In [2]:
original_image_list = ["../../images/kitten.jpg",
                       "../../images/dog.jpg"]

In [3]:
# Make a queue of filenames including all the images specified
filename_queue = tf.compat.v1.train.string_input_producer(original_image_list)

# Read an entire image file
image_reader = tf.compat.v1.WholeFileReader()

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.map(tf.read_file)`.


In [5]:
with tf.compat.v1.Session() as sess:
    # Coordinate the loading of image files
    coord = tf.train.Coordinator()
    threads = tf.compat.v1.train.start_queue_runners(sess=sess, coord=coord)

    image_list = []
    for i in range(len(original_image_list)):
        # Read a whole file from the queue, the first returned value in the tuple is the
        # filename which we are ignoring
        _, image_file = image_reader.read(filename_queue)

        # Decode the image as a JPEG file, this will return it into a Tensor which we can
        # then use in training
        image = tf.image.decode_jpeg(image_file)

        # Get a tensor of resized images
        image = tf.compat.v1.image.resize_images(image, [224, 224])
        image.set_shape((224, 224, 3))

        # Perform some image transformations 
        image = tf.image.flip_up_down(image)
        image = tf.image.central_crop(image, central_fraction=0.5)

        # Get an image tensor and print its value
        image_array = sess.run(image)
        print(image_array.shape)

        # Converts a numpy array of the kind (224,224,3) to a tensor of shape (224,224,3)
        image_tensor = tf.stack(image_array)

        print(image_tensor)
        image_list.append(image_tensor)
    
    # Finish off the filename queue coordinator
    coord.request_stop()
    coord.join(threads=threads)

    # Converts all tensors to a single tensor with a 4th dimension
    # 4 images of (224, 224, 3) can be accessed as (0, 224, 224, 3), (1,224,224, 3)...(3, 224,224, 3)
    images_tensor = tf.stack(image_list)
    print(images_tensor)

    index = 0
    # Write image summary
    summary_writer = tf.compat.v1.summary.FileWriter("ImageTransformationsWithCoordinator", graph=sess.graph)

    # Write out all the images in one go
    summary_str = sess.run(tf.compat.v1.summary.image("images", images_tensor))
    summary_writer.add_summary(summary_str)

    summary_writer.close()


(112, 112, 3)
Tensor("stack_3:0", shape=(112, 112, 3), dtype=float32)
(112, 112, 3)
Tensor("stack_4:0", shape=(112, 112, 3), dtype=float32)
Tensor("stack_5:0", shape=(2, 112, 112, 3), dtype=float32)
